# DATA COLLECTION

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip3 install twint
!pip install tweepy
!pip install -U textblob
!pip install emoji --upgrade
!pip install --upgrade pip
!pip install wordcloud
!pip install nest_asyncio

In [ ]:
##ADD THIS INSTALL
!pip3 install --user --upgrade git+https://github.com/yunusemrecatalcam/twint.git@twitter_legacy2

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
import twint
import tweepy
import csv
import pandas as pd
import random


# DATA IMPORTATION (At this stage you should have already cleaned your data) 
You can't do data clean with this file

In [ ]:
#Import your data in the next cell# Use example pd.read_csv("COVID-19INNOVATION.csv") if your file is CSV 
#and use pd.read_excel("COVID-19INNOVATION.xlsx")
#the df is the title of your data. You have to change this in accordance to whatever title you used and make sure to replace df in all the cell with your title 

In [ ]:
df= pd.read_csv("COVID-19INNOVATION.csv")

In [ ]:
df.head()

In [ ]:
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in df.cleaned_tweets: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 1000, height = 800, 
                background_color='white', colormap='Set2', 
                collocations=False, 
                stopwords = stopwords, 
                min_font_size = 12).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (10, 10), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

# Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
def getSubjectivity(text):
    return TextBlob( str(text)).sentiment.subjectivity

def getPolarity(text):
    return TextBlob( str(text)).sentiment.polarity

In [ ]:
df.dropna(subset=['cleaned_tweets'], inplace = True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['Subjectivity'] = df['cleaned_tweets'].apply(getSubjectivity)
df['Polarity'] = df['cleaned_tweets'].apply(getPolarity)
df.head()

In [ ]:
# Create a function to compute negative (-1), neutral (0) and positive (+1) analysis
def get_Polarity_Analysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'
def get_Subjectivity_Analysis(score):
    if score >  0:
      return 'Opinion'
    else:
      return 'Fact'

df['Analysis_Polarity'] = df['Polarity'].apply(get_Polarity_Analysis)

df['Analysis_Subjectivity'] = df['Subjectivity'].apply(get_Subjectivity_Analysis)

# Show the dataframe
df.head()



In [ ]:
df.to_csv("PolaritySubjectivityInnovation.csv")

In [ ]:
df.info()

# SENTIMENT ANALYSIS

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(font_scale=2)

In [ ]:
plt.figure(figsize=(15,10)) 

# plt.style.use('seaborn-pastel')

plt.scatter(df['Polarity'], df['Subjectivity'], c=df['Polarity'], s=100, cmap='RdYlGn') 

plt.xlim(-1.1, 1.1)
plt.ylim(-0.1, 1.1) 
plt.title('Sentiment Analysis') 
plt.xlabel('Polarity') 
plt.ylabel('Subjectivity') 
plt.show(),

In [ ]:
# Show the value counts
df['Analysis_Polarity'].value_counts()

In [ ]:
# Plotting and visualizing the counts
plt.figure(figsize=(15,10)) 

plt.title('Polarity Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis_Polarity'].value_counts().plot(kind = 'bar')
plt.show()

# Sentiment Analysis Pie Chart

In [ ]:
positive_polarity = [p for p in df['Polarity'] if p>0]
negative_polarity = [n for n in df['Polarity'] if n<0]
neutral_polarity = [r for r in df['Polarity'] if r==0]

total_size = len(positive_polarity) + len(negative_polarity) + len(neutral_polarity)
n_size = len(negative_polarity)/total_size
p_size = len(positive_polarity)/total_size
r_size = len(neutral_polarity)/total_size

labels = ['Neutral tweets', 'Positive tweets', 'Negative tweets']
sizes = [r_size, p_size, n_size]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
	        shadow=False, startangle=90)
ax1.axis('equal')
plt.show()

In [ ]:
# Plotting and visualizing the counts
plt.figure(figsize=(15,10)) 

plt.title('Subjectivity Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis_Subjectivity'].value_counts().plot(kind = 'bar')
plt.show()

In [ ]:
# Show the value counts
df['Analysis_Subjectivity'].value_counts()

# TOPIC MODELING

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(df['cleaned_tweets'].values.astype('U'))
print(dtm_tf.shape)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df['cleaned_tweets'].values.astype('U'))
print(dtm_tfidf.shape)

In [ ]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components =8, random_state=50)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components =8, random_state=50)
lda_tfidf.fit(dtm_tfidf)

In [ ]:
#for i,topic in enumerate(lda_tf.components_):
#print([tfidf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
#print('\n')

In [ ]:
for i,topic in enumerate(lda_tf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-30:]])
    print('\n')

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

In [ ]:
topic_values = lda_tf.transform(dtm_tf)
topic_values.shape

In [ ]:
df.head()

In [ ]:
# for TF DTM
#lda_tf = LatentDirichletAllocation(n_components =6, random_state=42)
#lda_tf.fit(dtm_tf)
# for TFIDF DTM
#lda_tfidf = LatentDirichletAllocation(n_components =6, random_state=42)
#lda_tfidf.fit(dtm_tfidf)

In [ ]:
df.columns

In [ ]:
df_1=df.replace({0:'Webinar',1:'Learning',2:'Rwanda',3:'Healthcare',4:'Technology',5:'Coronavirus',6:'Medical',7:'Challenge'})

In [ ]:
df_1['Topic'] = topic_values.argmax(axis=1)

In [ ]:
df['Topic'] = topic_values.argmax(axis=1)

In [ ]:
df_1.head()

In [ ]:
df_1.columns

In [ ]:
import seaborn as sns


plt.figure(figsize=(40,25)) 

g=sns.lmplot(x="Polarity", y="Subjectivity", hue='Topic', data=df, fit_reg=False, legend=False,palette="GnBu_d", col='Topic', legend_out=True)
 
# # Move the legend to an empty part of the plot
# plt.legend(loc='lower right')
 
plt.show()

In [ ]:
df2 = df_1.groupby(['Topic'])['Analysis_Polarity'].value_counts().unstack('Topic').transpose()

df2

In [ ]:
df2.info()

In [ ]:
df2['Total'] = df2.sum(axis=1)

In [ ]:
df2.columns

In [ ]:
for i in df2:
    df2[i] = round(df2[i]*100/df2.Total)

df2

In [ ]:
df2=df2.fillna(0)

In [ ]:
#Conduct Polarity Topic Analysis using Tableau

df.to_excel("Polarity_Topic Modeling.xlsx")

In [ ]:
df2= df2.drop(['Total'], axis=1)

In [ ]:
#Plotting and visualizing the counts
plt.figure(figsize=(20,15)) 


topic = ['Webinar','Learning','Rwanda','Healthcare','Technology','Coronavirus','Medical','Challenge']
sentiment = ['Negative', 'Neutral', 'Positive']
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('%')
p1=plt.bar(topic,df2['Negative'], color='#23646e')
p2=plt.bar(topic,df2['Neutral'], color='#419dab', bottom=df2['Negative'])
p3=plt.bar(topic,df2['Positive'], color='#6bbbc7', bottom=df2['Neutral']+df2['Negative'])
plt.xticks(topic, rotation=90)
plt.xlabel("Topic")
plt.legend((p1[0], p2[0], p3[0]),('Negative', 'Neutral', 'Positive'),fontsize=12, ncol=4, framealpha=0, fancybox=True, loc='upper center')

plt.show()


In [ ]:
df2.columns

In [ ]:
df=df.replace({0:'Webinar',1:'Learning',2:'Rwanda',3:'Healthcare',4:'Technology',5:'Coronavirus',6:'Medical',7:'Challenge'})

In [ ]:
df.head()

# Topic Counts

In [ ]:
# Show the value counts
df['Topic'].value_counts()

In [ ]:
plt.figure(figsize=(20,15)) 
sns.countplot(x = 'Topic',data = df, palette = 'viridis')


# TREND ANALYSIS

In [ ]:
# Create date range with frequency of a day
rng = pd.date_range(start='05/10/2020', end ='01/03/2020',freq='D')



In [ ]:

df3 = df.groupby(['date','Topic'])['cleaned_tweets'].count().reset_index()

In [ ]:
df3

In [ ]:
pivoted = df3.pivot( index='date',columns='Topic', values='cleaned_tweets').reset_index()

In [ ]:
pivoted

In [ ]:
pivoted=pivoted.fillna(0)

In [ ]:
pivoted.to_csv("tweets_by_date.csv")

In [ ]:
import matplotlib.dates as mdates

plt.figure(figsize=(20,15)) 

# gca stands for 'get current axis'
ax = plt.gca()

pivoted.plot(kind='line', x='date', y='Challenge', label='Challenge',ax=ax)
pivoted.plot(kind='line', x='date', y='Coronavirus', label='Coronavirus',ax=ax)
pivoted.plot(kind='line', x='date', y='Healthcare', label='Healthcare',ax=ax)
pivoted.plot(kind='line', x='date', y='Learning', label='Learning',ax=ax)
pivoted.plot(kind='line', x='date', y='Medical', label='Medical',ax=ax)
pivoted.plot(kind='line', x='date', y='Rwanda', label='Rwanda',ax=ax)
pivoted.plot(kind='line', x='date', y='Technology', label='Technology',ax=ax)
pivoted.plot(kind='line', x='date', y='Webinar', label='Webinar',ax=ax)


# ax.xaxis_date()
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'));

plt.legend()
plt.show()



# CLUSTER ANALYSIS WITH HEATMAP

In [ ]:
# # Preparing a corpus for analysis and checking first 10 entries

corpus=[]
a=[]
for i in range(len(df['cleaned_tweets'])):
    a=df['cleaned_tweets'][i]
    corpus.append(a)
        
print(corpus[0:10])

In [ ]:
import os
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# init_notebook_mode(connected=True) #do not miss this line

In [ ]:
# removing common words and tokenizing
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

dictionary = corpora.Dictionary(texts)

# print(dictionary)
# print(dictionary.token2id)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

In [ ]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

In [ ]:
total_topics = 2

lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

#Show first n important word in the topics:
lda.show_topics(total_topics,10)

In [ ]:

total_topics = 3

lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

#Show first n important word in the topics:
lda.show_topics(total_topics,10)

In [ ]:
data_lda = {i: OrderedDict(lda.show_topic(i,10)) for i in range(total_topics)}
data_lda

In [ ]:

df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

In [ ]:
df_lda

In [ ]:
try_lda=df_lda.T

In [ ]:
try_lda

In [ ]:
g=sns.clustermap(df_lda.corr(), center=0, standard_scale=1, cmap="coolwarm", metric='cosine', linewidths=.75, figsize=(15, 15), col_cluster=False)
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)

plt.show()
#plt.setp(ax_heatmap.get_yticklabels(), rotation=0)  # For y axis

In [ ]:

# pyLDAvis.enable_notebook()
# panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
# panel

# CORRELATION 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
%matplotlib inline

In [ ]:
dataset = pd.read_csv('tweets_by_date.csv')


# Statistical Summary

In [ ]:
# Statistical summary
dataset.describe().transpose()

In [ ]:
# Identify continuous variables
print(dataset.select_dtypes(['float']).columns)

In [ ]:

##Next, I want to compare the relationship (Correlation) between the identified topics in the dataset.

sns.pairplot(dataset[['Challenge', 'Coronavirus', 'Healthcare', 'Learning', 'Medical',
       'Rwanda', 'Technology', 'Webinar']])

plt.show()

In [ ]:
#This is another visual representation of Correlation Matrix to use
## SELECT COLORS https://python-graph-gallery.com/92-control-color-in-seaborn-heatmaps/


corrMatrix = pivoted.corr()

sns.set(font_scale=1)
sns.heatmap(corrMatrix, annot=False,
            xticklabels=corrMatrix.columns,
            yticklabels=corrMatrix.columns,
            cmap='RdBu_r'
            )
plt.title('Correlation matrix')
plt.show()

In [ ]:
#This is another visual representation of Correlation Matrix to use
sns.set(font_scale=1)
sns.heatmap(corrMatrix, annot=True, linewidth = 0.5, cmap='Spectral_r',square=False, vmin = -1, vmax = 1)
plt.title('Correlation matrix')

plt.show()

# CLUSTER ANALYSIS HEATMAP

In [ ]:
# Create cluter map
sns.clustermap(corrMatrix, annot=True, linewidth = 0.5, cmap='RdBu_r',square=False, vmin = -1, vmax = 1)
plt.title('Cluster Map')

plt.show()